In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import io
import os

# General purpose libraries
import tqdm
import imageio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# from sewar.full_ref import ssim, msssim, psnr

from skimage.measure import compare_ssim as ssim

# Framework modules
from helpers import coreutils, plotting

In [ ]:
dirname = './data/clic/'
images = coreutils.listdir(dirname)
images = images[:1]
print('Found {} images'.format(len(images)))

In [ ]:
quality_levels = np.arange(95, 10, -5)

df = pd.DataFrame(columns=['image_id', 'filename', 'codec', 'quality', 'ssim', 'psnr', 'bpp'])

with tqdm.tqdm(total=len(images) * len(quality_levels), ncols=120, desc='JPEG') as pbar:
    
    for image_id, filename in enumerate(images):

        # Read the original image
        image = imageio.imread(os.path.join(dirname, filename))
        
        for qi, q in enumerate(quality_levels):            
            
            # Compress to & read from memory
            s = io.BytesIO()
            imageio.imsave(s, image, format='jpg', quality=q)
            image_compressed = imageio.imread(s.getvalue())
            
            df = df.append({'image_id': image_id,
                            'filename': filename,
                            'codec': 'jpeg',
                            'quality': q,
                            'ssim': ssim(image, image_compressed, multichannel=True),  # ssim(image, image_compressed), # msssim(image, image_compressed).real
                            'psnr': psnr(image, image_compressed),
                            'bpp': 8 * len(s.getvalue()) / image.shape[0] / image.shape[1]
                            }, ignore_index=True)
        
            pbar.set_postfix(image_id=image_id, quality=q)
            pbar.update(1)

In [ ]:
df

In [ ]:
df[['ssim', 'bpp']]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True)
sns.lineplot(x='bpp', y='ssim', data=df, markers=True, ax=axes[0])
sns.lineplot(x='bpp', y='psnr', data=df, markers=True, ax=axes[1])

In [ ]:
# Read the original image
image = imageio.imread(os.path.join(dirname, filename))

# Compress to & read from memory
s = io.BytesIO()
imageio.imsave(s, image, format='jpg', quality=10)
image_compressed = imageio.imread(s.getvalue())

img_bytes = len(s.getvalue())
img_bpp = 8 * img_bytes / image.shape[0] / image.shape[1]
img_ssim = msssim(image, image_compressed).real
img_psnr = psnr(image, image_compressed)

fig = plotting.imsc((image, image_compressed), ['Original {}', 'JPEG {:.0f} kB / bpp: {:.2f} -> {:.1f} dB / {:.2f}'.format(img_bytes / 1024, img_bpp, img_psnr, img_ssim)], figwidth=20)

In [ ]:
imageio.help('jp2')